<a href="https://colab.research.google.com/github/kkmax1015/Kaggle/blob/main/Titanic/2_%E6%AC%A0%E6%90%8D%E5%80%A4%E3%81%A8%E3%82%AB%E3%83%86%E3%82%B4%E3%83%AA%E3%83%BC%E5%A4%89%E6%95%B0%E3%81%AE%E5%8F%96%E3%82%8A%E6%89%B1%E3%81%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Googleドライブへのマウント
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/Titanic/'
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Titanic
1_SampleNoteBook_LogisticRegression	  data
2_欠損値とカテゴリー変数の取り扱い.ipynb  output


# 目標：欠損値とカテゴリー変数の取り扱いを知る

前回のノートブックではNULLが存在する特徴量や文字が入っている特徴量は全て削除したが、無条件に削除してしまっては予測に有用な特徴量も削ってしまうかもしれません。

そこで、今回はNULLや数値以外の特徴量を使用して学習した場合の精度を確認してみます。

###１.欠損値の補完

In [2]:
# PandasをImportする
import pandas as pd

In [3]:
# CSVデータをPandasデータフレームへ読み込む
train = pd.read_csv("./data/train.csv");
test = pd.read_csv("./data/test.csv");
submit = pd.read_csv("./data/gender_submission.csv");

In [4]:
# それぞれのデータの件数を算出する
print(len(train), len(test),len(submit))

891 418 418


In [5]:
# トレーニング用データの先頭5行を表示
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# テスト用データの先頭5行を表示
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


データを読み込んだら、まずトレーニング用データとテストデータを結合して特徴量別のNULLの数を確認する。

(トレーニングデータ、テストデータそれぞれでしかNULLではない項目がいる可能性がある為、結合して確認する。)

In [7]:
# トレーニング用データから'Survived'を削除したものをdataへ代入（テストデータに無い為）
# dropは元データ（この場合はtrain）からはデフォルトでは削除されない。元データから削除する場合は引数 inplaseをtrueに設定する。
# ex)train.drop('Survived',axis=1,inplace=True)
data = train.drop('Survived',axis=1)
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
# inplaseをtrueに設定していない為、trainからは'Survived'は削除されていない。
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
# dataへテストデータをマージ
data = pd.concat([data,test],join='inner')
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
# Name、Ticket、Cabinはカテゴリー変数ではない為削除する。PassengerIdもデータを一意にする識別子であり、予測に関係ない可能性が高いので今回は削除する。
# dataから削除してしまって良いので、inplaseをtrueに設定する。
data.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)
data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [11]:
# dataの特徴量別のNULLの数を確認してみる
data.isnull().sum()

Pclass        0
Sex           0
Age         263
SibSp         0
Parch         0
Fare          1
Embarked      2
dtype: int64

ここまでの処理でデータ全体でAge、Fare、CabinへNULLが存在することが分かった。


各項目の意味は以下の通り。

|変数名|定義|値の意味|
|-|-|-|
|Pclass|チケットクラス|1 = 1等室、2 = 2等室、3 = 3等室|
|Sex|性別|
|Age|年齢|
|SibSp|タイタニックに同乗している兄弟/配偶者の数|
|Parch|タイタニックに同乗している親/子供の数|
|Fare|旅客運賃|
|Embarked|出港地（タイタニックへ乗った港）|C =シェルブール、Q =クイーンズタウン、S =サウサンプトン|

欠損している項目の意味を考慮しながら補完する方針を考えていく。

### 1.1 欠損値の扱いについて

欠損値の扱いについては大まかに以下の3つの方針のいずれかで対応する。



*   欠損値のまま取り扱う
*   代理データへ置き換え
*   欠損値を予測する



#### 欠損値まま取り扱う

何らかの要因で欠損値となっているのだと考え、「欠損している」という事実そのものが有用な情報であると考える。その為、あえて欠損させたままにする。

欠損値があっても学習できるライブラリの場合はそのままにし、欠損値を扱えないライブラリの場合は「-9999」等、通常取りうる範囲外の値を設定することで欠損値をそのままにした場合と近い状態にすることが出来る。

#### 代理データへ置き換え

何らかの代理データへ置き換える。代表的な手法は以下の5つ。データの特性を踏まえた検討が必要。



*   直前のデータの値で補完
*   直後のデータの値で補完
*   中央値で補完
*   平均値で補完
*   最も多い値で補完


#### 欠損値を予測する

他の特徴量から欠損値を予測する機械学習モデルを作成し、予測した値を設定する。

今回は以上の方針の中から補完の方針を決めていく。

In [12]:
# Ageの欠損値補完
# 今回は中央値で補完する
data['Age'].fillna(data['Age'].median(), inplace=True)

In [13]:
# Fareの欠損値補完
# 今回は平均値で補完する
import numpy as np
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)

In [14]:
# Embarkedの欠損値補完
# 最も多い数を確認する。
data['Embarked'].value_counts()

S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [15]:
# 最も多いSで補完
data['Embarked'].fillna('S', inplace=True)

In [16]:
# dataの特徴量別のNULLの数を確認してみる
data.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [17]:
data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


以上で欠損値の対応が完了し、学習可能な状態となった。

### 2.カテゴリー変数の取り扱い

欠損値の対応は完了したが、前回見たように文字のままでは学習をさせることが出来ない為、SexとEmbarkedを変換する。

カテゴリー変数の取り扱いはいくつかあるが、今回は単純に整数へ置き換える「label encoding」という手法で対応する。

Sex：male→0　female→1　へ変換

Embarked：へ変換

In [18]:
data['Sex'].replace(['male','female'], [0, 1], inplace=True)
data['Embarked'].replace(['S','C','Q'], [0, 1, 2], inplace=True)

In [19]:
data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22.0,1,0,7.2500,0
1,1,1,38.0,1,0,71.2833,1
2,3,1,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,3,0,35.0,0,0,8.0500,0


### 3.モデルの作成

今回作成したデータを用いて機械学習モデルを作成する。

In [20]:
# トレーニングデータ、テストデータの作成
# X_train、X_testは欠損値、カテゴリー変数の対応をしたdataから作成する。
y_train = train['Survived']
X_train = data[:len(train)]
X_test = data[len(train):]

In [21]:
# sklearnのロジスティック回帰をImport
from sklearn.linear_model import LogisticRegression

# ロジスティック回帰モデルの作成(初期化)
# random_state：初期化パラメータ。指定することで学習結果を固定することが出来る。
clf = LogisticRegression(random_state=0)

In [22]:
# 学習
clf.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
# 予測値の算出
y_pred = clf.predict(X_test)

In [24]:
# 提出ファイルの作成
sub = submit
sub['Survived'] = y_pred
sub.to_csv('./output/submission.csv', index=False)